<a href="https://colab.research.google.com/github/sahil-verma-9696/py-speech-emotion-recognition/blob/main/SER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
import pandas as pd

sentences = [
    "सूरज एक अच्छा विद्यार्थी है।",
    "गणित का प्रश्न पत्र कठिन था।",
    "मुझे अच्छे अंक लाने हैं।",
    "रीना पढ़ाई में सबकी मदद करती है।",
    "विद्यार्थी आंदोलन कर रहे थे।",
    "मैंने यह पाठ पूरा नहीं पढ़ा है।",
    "अच्छा विद्यार्थी अहंकार से दूर रहता है।",
    "विद्यार्थियों को ईमानदार होना चाहिए।",
    "आप सभी मन लगाकर पढ़िए।",
    "विद्यार्थी परिणाम का इंतजार कर रहे थे।"
]

emotions = [
    "Anger",
    "disgust",
    "fear",
    "happy",
    "neutral",
    "sad",
    "sarcastic",
    "surprise"
]

# Generate all combinations
data = [{"text": sentence, "emotion": emotion} for sentence in sentences for emotion in emotions]

# Save to CSV
df = pd.DataFrame(data)
df.to_csv("sentence_emotion_mapping.csv", index=False, encoding="utf-8-sig")
print("✅ CSV created: sentence_emotion_mapping.csv")


✅ CSV created: sentence_emotion_mapping.csv


In [ ]:
df = pd.read_csv("sentence_emotion_mapping.csv")
# print(df)/content/drive/MyDrive/dataset/my Dataset

In [ ]:
import os
import csv

# Replace with your dataset root
base_dir = '/content/drive/MyDrive/dataset/my Dataset/1'

# Output CSV file path
output_csv = 'audio_mapping.csv'
sentences = [
    "सूरज एक अच्छा विद्यार्थी है।",
    "गणित का प्रश्न पत्र कठिन था।",
    "मुझे अच्छे अंक लाने हैं।",
    "रीना पढ़ाई में सबकी मदद करती है।",
    "विद्यार्थी आंदोलन कर रहे थे।",
    "मैंने यह पाठ पूरा नहीं पढ़ा है।",
    "अच्छा विद्यार्थी अहंकार से दूर रहता है।",
    "विद्यार्थियों को ईमानदार होना चाहिए।",
    "आप सभी मन लगाकर पढ़िए।",
    "विद्यार्थी परिणाम का इंतजार कर रहे थे।"
]

# Dummy function for transcription (replace this with Whisper or any STT engine)
def transcribe_audio(filename):
    core = filename.rsplit('.', 1)[0]   # '1.1.anger-01'
    last_part = core.rsplit('-', 1)[-1] # '01'

    print(sentences[int(last_part)-1])

    return sentences[int(last_part)-1]

# Collect data
data = []

for root, _, files in os.walk(base_dir):
    for file in files:
        if file.endswith('.wav'):
            file_path = os.path.join(root, file)
            parts = file_path.split(os.sep)

            try:
                # Assuming emotion is always at index -2: session1/anger/filename
                emotion_name = parts[-2]
                spoken_text = transcribe_audio(file_path)

                data.append([file_path, emotion_name, spoken_text])
            except IndexError:
                print(f"Skipping {file_path} due to unexpected format")

# Write to CSV
with open(output_csv, 'w', newline='',encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['file_path', 'emotion_name', 'spoken_text'])
    writer.writerows(data)

print(f"CSV saved to {output_csv}")


8 * 5 * 8 * 10 = 3200
person.session.emotion-sentence


In [ ]:
import os
from googleapiclient.discovery import build
from google.colab import auth
import google.auth

# Authenticate and build the Drive service
auth.authenticate_user()
creds, _ = google.auth.default()
drive_service = build('drive', 'v3', credentials=creds)

def get_folder_id_by_path(path):
    path_parts = path.strip('/').split('/')
    folder_id = 'root'

    for part in path_parts:
        query = f"'{folder_id}' in parents and name = '{part}' and mimeType = 'application/vnd.google-apps.folder' and trashed = false"
        result = drive_service.files().list(q=query, fields="files(id, name)").execute()
        files = result.get('files', [])
        if not files:
            raise Exception(f"Folder '{part}' not found under parent ID {folder_id}")
        folder_id = files[0]['id']

    return folder_id

# Get source folder ID
source_path = 'dataset'
source_id = get_folder_id_by_path(source_path)
print(f"Source Folder ID: {source_id}")


Source Folder ID: 1Od_8WezSCPXF18LWKwTe7GDyIQsGwQAd


In [ ]:
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib
from google.colab import auth
from googleapiclient.discovery import build
import google.auth

# Authenticate and build Drive API client
auth.authenticate_user()
creds, _ = google.auth.default()
drive_service = build('drive', 'v3', credentials=creds)

SOURCE_FOLDER_ID = '1Ac57ZNbwjy1fQ1jZ-43x02tXAYtc1ppf'  # Your nested folder ID
DEST_FOLDER_ID = '1Od_8WezSCPXF18LWKwTe7GDyIQsGwQAd'  # Replace after getting it

def list_all_files_recursive(folder_id):
    files = []
    query = f"'{folder_id}' in parents and trashed = false"
    results = drive_service.files().list(q=query, fields="files(id, name, mimeType)").execute()

    for file in results.get('files', []):
        if file['mimeType'] == 'application/vnd.google-apps.folder':
            files.extend(list_all_files_recursive(file['id']))
        else:
            files.append(file)
    return files

def move_file(file_id, new_parent_id):
    file = drive_service.files().get(fileId=file_id, fields='parents').execute()
    previous_parents = ",".join(file.get('parents'))
    drive_service.files().update(
        fileId=file_id,
        addParents=new_parent_id,
        removeParents=previous_parents,
        fields='id, parents'
    ).execute()

# Flattening begins
files_to_move = list_all_files_recursive(SOURCE_FOLDER_ID)
print(f"Found {len(files_to_move)} files to move.")

for file in files_to_move:
    print(f"Moving: {file['name']}")
    move_file(file['id'], DEST_FOLDER_ID)

print("✅ All files moved and folder flattened.")

In [ ]:
path = '/gdrive/MyDrive/ser_ds/audios'
import os

In [ ]:

sentences = [
    "सूरज एक अच्छा विद्यार्थी है।",
    "गणित का प्रश्न पत्र कठिन था।",
    "मुझे अच्छे अंक लाने हैं।",
    "रीना पढ़ाई में सबकी मदद करती है।",
    "विद्यार्थी आंदोलन कर रहे थे।",
    "मैंने यह पाठ पूरा नहीं पढ़ा है।",
    "अच्छा विद्यार्थी अहंकार से दूर रहता है।",
    "विद्यार्थियों को ईमानदार होना चाहिए।",
    "आप सभी मन लगाकर पढ़िए।",
    "विद्यार्थी परिणाम का इंतजार कर रहे थे।"
]

emotions = [
    "Anger",
    "disgust",
    "fear",
    "happy",
    "neutral",
    "sad",
    "sarcastic",
    "surprise"
]

In [ ]:
import os
import pandas as pd
import traceback

In [ ]:
arr = []
cnt= 1
for root, _, files in os.walk(path):
    for file in files:
        if file.endswith('.wav'):
            file_path = os.path.join(root, file)
            parts = file_path.split(os.sep)
            try:
              arr.append({'id': cnt, 'file_path': file_path, 'emotion': parts[-1].split('.')[-2][:-2] , 'text': sentences[int(parts[-1].split('.')[-2][-2:]) - 1] })
              cnt+=1
            except Exception as e:
              print(file_path)
              print(parts)
              print( parts[-1].split('.')[-2].split('-')[0] )
              print( parts[-1].split('.')[-2].split('-')[1] )
              print(sentences[int(parts[-1].split('.')[-2].split('-')[1]) - 1])

              print(traceback.format_exc())
              print(f"Skipping {file_path} due to unexpected format")

df = pd.DataFrame(arr)
df.to_csv("/gdrive/MyDrive/ser_ds/SER_ds_csv.csv", index=False, encoding="utf-8-sig")
print("✅ CSV created: sentence_emotion_mapping.csv")

✅ CSV created: sentence_emotion_mapping.csv


In [ ]:
# audio file link - https://drive.google.com/drive/folders/1Od_8WezSCPXF18LWKwTe7GDyIQsGwQAd?usp=drive_linkm
# ds link - https://drive.google.com/file/d/1OCDGB2qjIVVQ-3DiSUZPiQmBMJDTW8r9/view?usp=drive_link

In [ ]:

df


In [ ]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

# Extracting the features from audio

In [ ]:
# --------------------------
# 1. Install Dependencies
# --------------------------
!pip install librosa soundfile praat-parselmouth numpy scipy

# --------------------------
# 2. Import Required Libraries
# --------------------------
import librosa
import numpy as np
import parselmouth
import soundfile as sf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 96.7 MB/s eta 0:00:00


In [ ]:

# Automatically get filename
audio_path = '/content/drive/MyDrive/dataset/1.1.anger-01.wav'

# --------------------------
# 5. Feature Extraction Function
# --------------------------
def extract_all_features(audio_path):
    import librosa
    import numpy as np
    import parselmouth

    y, sr = librosa.load(audio_path, sr=None)

    # MFCC
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).mean(axis=1)

    # Chroma
    chroma = librosa.feature.chroma_stft(y=y, sr=sr).mean(axis=1)

    # Spectral Contrast
    spec_contrast = librosa.feature.spectral_contrast(y=y, sr=sr).mean(axis=1)

    # Tonnetz
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr).mean(axis=1)

    # Zero Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(y).mean()

    # Spectral Features
    spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
    spec_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr).mean()
    spec_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr).mean()

    # Root Mean Square Energy
    rmse = librosa.feature.rms(y=y).mean()

    # Pitch
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    pitch_values = pitches[pitches > 0]
    pitch_mean = pitch_values.mean() if len(pitch_values) > 0 else 0

    # Parselmouth for jitter, shimmer, HNR
    snd = parselmouth.Sound(audio_path)
    point_process = parselmouth.praat.call(snd, "To PointProcess (periodic, cc)", 75, 500)

    # Jitter
    jitter_local = parselmouth.praat.call(point_process, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)

    # Shimmer
    shimmer_local = parselmouth.praat.call([snd, point_process], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)

    # Harmonics-to-Noise Ratio (HNR)
    harmonicity = parselmouth.praat.call(snd, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    hnr = parselmouth.praat.call(harmonicity, "Get mean", 0, 0)

    # Combine all features
    feature_vector = np.hstack([
        mfccs,
        chroma,
        spec_contrast,
        tonnetz,
        zcr,
        spec_centroid,
        spec_bandwidth,
        spec_rolloff,
        rmse,
        pitch_mean,
        jitter_local,
        shimmer_local,
        hnr
    ])

    return feature_vector

# --------------------------
# 6. Extract and Print Features
# --------------------------
features = extract_all_features(audio_path)
print("Extracted feature vector (length={}):".format(len(features)))
print(features)


Extracted feature vector (length=47):
[-3.98995758e+02  9.67277679e+01 -3.38579154e+00  1.80757847e+01
 -2.97621422e+01 -8.93530083e+00 -4.56137323e+00 -1.22554865e+01
 -1.89918671e+01 -1.13129320e+01 -1.63695412e+01 -1.77026291e+01
 -8.24666977e+00  5.50655305e-01  4.30559635e-01  3.31397891e-01
  2.76145458e-01  3.28886867e-01  3.89419079e-01  3.93849581e-01
  4.18670207e-01  3.49334031e-01  4.01092172e-01  4.44719464e-01
  5.23074687e-01  2.30516856e+01  1.63047989e+01  1.79890790e+01
  1.76174481e+01  1.70567081e+01  2.01933760e+01  1.89000991e+01
  1.24771587e-02  2.47011746e-02  1.15165342e-02  3.50669621e-02
 -9.10471638e-03  7.29467026e-03  1.05894697e-01  1.62527663e+03
  1.61038684e+03  3.31017287e+03  1.57346204e-02  9.48319153e+02
  2.20069688e-02  1.01566960e-01  1.33835968e+01]


# New Section

In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 2. Install dependencies
!pip install librosa tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 114.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 123.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.7 MB/s eta 0:00:00


In [ ]:
# 3. Import required libraries
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Attention
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os

In [ ]:
# 4. Feature extraction function (MFCCs)
def extract_mfcc(file_path, max_len=300):
    y, sr = librosa.load(file_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mfcc = mfcc.T
    if len(mfcc) > max_len:
        mfcc = mfcc[:max_len]
    else:
        mfcc = np.pad(mfcc, ((0, max_len - len(mfcc)), (0, 0)))
    return mfcc


In [ ]:
# 5. Load your sample
file_path = "/content/drive/MyDrive/dataset/1.1.anger-01.wav"
X_sample = extract_mfcc(file_path)
X_sample = np.expand_dims(X_sample, axis=0)  # Shape: (1, time_steps, 40)



# Simulate label for demonstration (anger → class 0)
y_sample = tf.keras.utils.to_categorical([0], num_classes=4)  # Assume 4 emotions


In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling1D

# Input
input_layer = Input(shape=(X_sample.shape[1], 40))

# CNN
x = Conv1D(128, kernel_size=5, activation='relu')(input_layer)
x = MaxPooling1D(pool_size=2)(x)

# BiLSTM
x = Bidirectional(LSTM(64, return_sequences=True))(x)

# Attention
attention_out = Attention()([x, x])  # (batch_size, timesteps, features)

# Replace reduce_mean with a proper Keras layer
context_vector = GlobalAveragePooling1D()(attention_out)

# Output
output_layer = Dense(4, activation='softmax')(context_vector)

# Model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 300, 40)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 296, 128)  │     25,728 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 148, 128)  │          0 │ conv1d_1[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 148, 128)  │     98,816 │ max_pooling1d_1[… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_1         │ (None, 148, 128)  │          0 │ bidirectional_1[… │
│ (Attention)         │                   │            │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ attention_1[0][0] │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 4)         │        516 │ global_average_p… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 125,060 (488.52 KB)

 Trainable params: 125,060 (488.52 KB)

 Non-trainable params: 0 (0.00 B)